In [7]:
import pandas as pd

# Load CSV (Ensure correct filename and check if the date column is named correctly)
df = pd.read_csv("COVID_US_cases.csv", parse_dates=["date"])

# Ensure lowercase column names to avoid key errors
df.columns = df.columns.str.lower()

# Create "Week Start Date" (Monday as the start of the week)
df["week_start_date"] = df["date"] - pd.to_timedelta(df["date"].dt.weekday, unit="D")

# Aggregate weekly data
weekly_df = df.groupby("week_start_date", as_index=False).agg({
    "new_confirmed": "sum",
    "new_deceased": "sum",
    "new_recovered": "sum",
    "new_tested": "sum",
    "cumulative_confirmed": "max",  # Cumulative numbers should not be summed, use max()
    "cumulative_deceased": "max",
    "cumulative_recovered": "max",
    "cumulative_tested": "max"
})

# Compute proportions (%) of total tested
weekly_df["confirmed_rate"] = (weekly_df["new_confirmed"] / weekly_df["cumulative_tested"]) * 100
weekly_df["deceased_rate"] = (weekly_df["new_deceased"] / weekly_df["cumulative_tested"]) * 100
weekly_df["recovered_rate"] = (weekly_df["new_recovered"] / weekly_df["cumulative_tested"]) * 100

# Compute mortality rate (%) (Deaths per Confirmed Cases)
weekly_df["mortality_rate"] = (weekly_df["new_deceased"] / weekly_df["new_confirmed"]) * 100

# Ensure there are no divide-by-zero errors
weekly_df.replace([float("inf"), -float("inf")], 0, inplace=True)
weekly_df.fillna(0, inplace=True)

# Reshape the DataFrame for Flourish (Long Format)
flourish_df = weekly_df.melt(id_vars=["week_start_date"], 
                              value_vars=["confirmed_rate", "deceased_rate", "recovered_rate"],
                              var_name="category", 
                              value_name="value")

# Rename columns for clarity
flourish_df.rename(columns={"week_start_date": "Week Start Date", "category": "Category", "value": "Value"}, inplace=True)

# Save the final transformed CSV
flourish_df.to_csv("covid_weekly_transformed.csv", index=False)

print("✅ Weekly transformed data saved as 'covid_weekly_transformed.csv' for Flourish.")




✅ Weekly transformed data saved as 'covid_weekly_transformed.csv' for Flourish.
